In [5]:
import faiss
import sys

modules = '/home/dbcloud/ym/CSPG/ANNS/modules'

if modules not in sys.path:
  sys.path.append(modules)

from binary_io import *
import anns as pui

In [6]:
dataset = [
  # 'sift1m', 'gist1m', 
  'deep1m', 'text2image1m', 'turing1m', 'gcd']

read_ways = [
  # 'fvecs_read', 'fvecs_read', 
  'fbin_read', 'fbin_read', 'fbin_read', 'fvecs_read']

bases = [
  # "/var/lib/docker/anns/dataset/sift1m/base.fvecs",
  # "/var/lib/docker/anns/dataset/gist1m/base.fvecs",
  "/var/lib/docker/anns/dataset/deep1b/base.1M.fbin",
  "/home/dbcloud/big-ann-benchmarks/data/text2image1B/base.1B.fbin.crop_nb_1000000",
  "/home/dbcloud/big-ann-benchmarks/data/MSTuringANNS/base1b.fbin.crop_nb_1000000",
  "/home/dbcloud/ym/CSPG/experiment/distribution/data/base.fvecs"
]

queries = [
  # "/var/lib/docker/anns/query/sift1m/query.fvecs",
  # "/var/lib/docker/anns/query/gist1m/query.fvecs",
  "/var/lib/docker/anns/dataset/deep1b/query.public.10K.fbin",
  "/home/dbcloud/big-ann-benchmarks/data/text2image1B/query.heldout.30K.fbin",
  "/home/dbcloud/big-ann-benchmarks/data/MSTuringANNS/testQuery10K.fbin",
  "/home/dbcloud/ym/CSPG/experiment/distribution/data/query.fvecs"
]

gts = [
  # "/var/lib/docker/anns/query/sift1m/gt.ivecs",
  # "/var/lib/docker/anns/query/gist1m/gt.ivecs",
  "/var/lib/docker/anns/dataset/deep1b/gt_1M.ibin",
  "/home/dbcloud/big-ann-benchmarks/data/text2image1B/gt100-heldout.30K.ivecs",
  "/home/dbcloud/big-ann-benchmarks/data/MSTuringANNS/testQuery10K_gt.ivecs",
  "/home/dbcloud/ym/CSPG/experiment/distribution/data/gt.ivecs"
]

output = '/home/dbcloud/ym/CSPG/experiment/output/distribution/'

In [7]:
def test_faiss_ivf(base, query, gt, rw, dataset):

  if rw == 'fbin_read':
    base = fbin_read(base)
    query = fbin_read(query)
  else:
    base = fvecs_read(base)
    query = fvecs_read(query)

  if dataset == 'deep1m':
    gt = ibin_read(gt)
  else:
    gt = ivecs_read(gt)

  nb, d = base.shape
  nq, ngt = gt.shape
  gt = gt.flatten().tolist()

  k = 10
  threads = 24
  faiss.omp_set_num_threads(threads)

  nlist = 1204

  quantizer = faiss.IndexFlatL2(d) 
  index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
  # Train the index
  index.train(base)
  # Add the base vectors to the index
  index.add(base)

  print(ds, nb, d, nq, nlist, k)

  out = open(output + dataset + '_ivf.csv', 'w')

  tm = pui.STimer()

  print('num_queries,nlist,nprobe,query_time,recall', file=out)
  for npb in range(10, 200+1, 10):
    index.nprobe = npb
    tm.reset()
    tm.start()
    D, I = index.search(query, k)
    tm.stop()
    query_time = tm.get_time()
    recall = pui.get_recall(k, ngt, gt, I)
    print(f"{nq},{nlist},{npb},{query_time},{recall}", file=out)

  out.close()

In [8]:
for ds, rw, base, query, gt in zip(dataset, read_ways, bases, queries, gts):
  test_faiss_ivf(base, query, gt, rw, ds)

deep1m 1000000 96 10000 1204 10
text2image1m 1000000 200 30000 1204 10
turing1m 1000000 100 10000 1204 10
gcd 1000000 96 1000 1204 10
